In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [225]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from functools import reduce

In [38]:
doc_to_title = {}
with open('data/docs_titles.tsv/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
#         print(line)
        data = line.strip().split('\t', 1)
#         print('data = ',data)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
# print(doc_to_title)
print (len(doc_to_title)) 
doc_to_title #словарь id документа: заголовок

28026


{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

In [39]:
import pandas as pd
train_data = pd.read_csv('data/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
traingroups_titledata #Словарь номер группы: список пар (Id_doc, заголовок, target) для заголовков этой группы

{1: [(15731, 'ВАЗ 21213 | Замена подшипников ступицы | Нива', 0),
  (14829,
   'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
   0),
  (15764,
   'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
   0),
  (17669, 'Классика 21010 - 21074', 0),
  (14852, 'Ступица Нива — замена подшипника своими руками', 0),
  (15458, 'ВАЗ 2110', 0),
  (14899, 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123', 0),
  (16879, 'Купить Подшипники и ступицы FAG (Страница 23)', 0),
  (16310,
   'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
   0),
  (15440,
   'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
   0),
  (16242,
   'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
   0),
  (16383, 'Ступицы - OLX.ua - страница 80', 0),
  (15580,
   'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIV

In [40]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train, y_train, groups_train)

[[4 4 3 ... 2 1 1]
 [8 7 7 ... 2 2 2]
 [2 2 2 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [6 2 1 ... 1 1 1]
 [3 2 2 ... 1 1 1]] [0 0 0 ... 0 0 0] [  1   1   1 ... 129 129 129]


In [41]:
from sklearn.linear_model import SGDClassifier
log_reg = SGDClassifier(loss='log')

In [42]:
scaler.fit(X_train)
log_reg.fit(scaler.transform(X_train), y_train)
y_pred = log_reg.predict(scaler.transform(X_test))

NameError: name 'scaler' is not defined

In [43]:
data = test_data
data['target'] = y_pred
data = data.drop(['group_id', 'doc_id'], axis=1)

# print(data.head(100))
data['target'] = data['target'].apply(lambda x: 0 if x == -1 else 1)
data.to_csv("submission.csv", index=False)
data

NameError: name 'y_pred' is not defined

In [222]:
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score

In [223]:
def flatten(x):
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str):
            result.extend(flatten(elem))
        else:
            result.append(elem)
    return result

In [ ]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [232]:
def combinations(params):
    
    list_ = []
    for value in params.values():
        list_.append(value)
        
    tmp = reduce(lambda x, y: list_concat(x, y), list_)
    
    res = []
    for elem in tmp:
        param_list = flatten(elem)
        param_dict = dict(zip(params.keys(), param_list))
        res.append(param_dict)
        
    return res

In [ ]:
def validation(X_train, y_train, model, folds = 5, **kwargs):
    folds = KFold(folds, shuffle = True)
    fold_generator = folds.split(X_train, target) 
    for train_index, test_index in fold_generator:
        

In [182]:
params = {'loss': ['log', 'reg'],
          'alpha': [0.01, 0.001, 0.0001],
          'test': [1, 2, 3, 4], 
          'test2': ['a', 'b']}

In [233]:
combinations(params)

[{'loss': 'log', 'alpha': 0.01, 'test': 1, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.01, 'test': 1, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.01, 'test': 2, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.01, 'test': 2, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.01, 'test': 3, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.01, 'test': 3, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.01, 'test': 4, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.01, 'test': 4, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.001, 'test': 1, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.001, 'test': 1, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.001, 'test': 2, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.001, 'test': 2, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.001, 'test': 3, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.001, 'test': 3, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.001, 'test': 4, 'test2': 'a'},
 {'loss': 'log', 'alpha': 0.001, 'test': 4, 'test2': 'b'},
 {'loss': 'log', 'alpha': 0.0001, 'test': 1, 'test2': 'a'},
 {'l

In [184]:
list_ = []
for value in params.values():
    list_.append(value)
list_

[['log', 'reg'], [0.01, 0.001, 0.0001], [1, 2, 3, 4], ['a', 'b']]

In [221]:
res4 = []
for elem in res3:
    res4.append(flatten(elem))
res4

[['log', 0.01, 1, 'a'],
 ['log', 0.01, 1, 'b'],
 ['log', 0.01, 2, 'a'],
 ['log', 0.01, 2, 'b'],
 ['log', 0.01, 3, 'a'],
 ['log', 0.01, 3, 'b'],
 ['log', 0.01, 4, 'a'],
 ['log', 0.01, 4, 'b'],
 ['log', 0.001, 1, 'a'],
 ['log', 0.001, 1, 'b'],
 ['log', 0.001, 2, 'a'],
 ['log', 0.001, 2, 'b'],
 ['log', 0.001, 3, 'a'],
 ['log', 0.001, 3, 'b'],
 ['log', 0.001, 4, 'a'],
 ['log', 0.001, 4, 'b'],
 ['log', 0.0001, 1, 'a'],
 ['log', 0.0001, 1, 'b'],
 ['log', 0.0001, 2, 'a'],
 ['log', 0.0001, 2, 'b'],
 ['log', 0.0001, 3, 'a'],
 ['log', 0.0001, 3, 'b'],
 ['log', 0.0001, 4, 'a'],
 ['log', 0.0001, 4, 'b'],
 ['reg', 0.01, 1, 'a'],
 ['reg', 0.01, 1, 'b'],
 ['reg', 0.01, 2, 'a'],
 ['reg', 0.01, 2, 'b'],
 ['reg', 0.01, 3, 'a'],
 ['reg', 0.01, 3, 'b'],
 ['reg', 0.01, 4, 'a'],
 ['reg', 0.01, 4, 'b'],
 ['reg', 0.001, 1, 'a'],
 ['reg', 0.001, 1, 'b'],
 ['reg', 0.001, 2, 'a'],
 ['reg', 0.001, 2, 'b'],
 ['reg', 0.001, 3, 'a'],
 ['reg', 0.001, 3, 'b'],
 ['reg', 0.001, 4, 'a'],
 ['reg', 0.001, 4, 'b'],
 ['reg',

In [219]:
def adjust_lists(list1, list2):
    res = list1
    res.append(list2)
    return res

In [220]:
test = [[[['reg', 0.0001], 4], 'a'], 'hh']

In [198]:
reduce(lambda x, y: adjust_lists(x, y), test)

[[['reg', 0.0001], 4], 'a', 'hh']

In [202]:
from compiler.ast import flatten

ModuleNotFoundError: No module named 'compiler'